In [1]:
%matplotlib inline

import glob
import os
import random
import json

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

from astropy.table import Table
import rubin_sim.maf.db as db
from rubin_sim.maf.utils import m52snr
import rubin_sim.maf.metrics as metrics
import rubin_sim.maf.plots as plots

import rubin_sim.maf.slicers as slicers
import rubin_sim.maf.metricBundles as metricBundles
from rubin_sim.utils import equatorialFromGalactic, hpid2RaDec, _healbin, healbin
from rubin_sim.utils import getPackageDir
#from rubin_sim.maf import KN_lc, KNePopMetric, generateKNPopSlicer, get_filename
import healpy as hp

In [3]:
##RUN TWICE

import sys
import importlib

if "MDwarfFlares" in sys.modules:
    del sys.modules["MDwarfFlares"]

import MDwarfFlares
importlib.reload(MDwarfFlares)

from MDwarfFlares import MDwarfFlareMetric, generateMDwarfFlareSlicer
print(MDwarfFlares.__file__)


/home/andra104/Stuff/AllTransient_MetricDetection/MDwarfFlares.py


In [4]:

# M Dwarf Flare detection parameters
dmin = 10   # Minimum distance in Mpc
dmax = 1000 # Max distance for M Dwarf Flares

cadence_nickname = 'baseline_v4.3.1_10yrs' # Used in output filenames

# Output file for M Dwarf Flare efficiency results
outfile = f"AllTransient_MetricDetection/efficiency_{cadence_nickname}_mdwarf.csv"

# Generate the M Dwarf Flare population slicer
slicer = generateMDwarfFlareSlicer(t_start=1, t_end=3652, seed=42)

# Extract the actual number of simulated events from the slicer (AFTER defining it)
n_events = len(slicer.slicePoints['peak_time'])

# Load Rubin cadence database
runName = 'baseline_v4.3.1_10yrs'
opsdb = db.OpsimDatabase(f'../AllTransient_MetricDetection/{runName}.db')

outDir = "AllTransient_MetricDetection/Metric_temp"
resultsDb = db.ResultsDb(outDir=outDir)

print(f"Total M Dwarf Flare events simulated: {n_events}")
print(f"First few peak times (days): {slicer.slicePoints['peak_time'][:5]}")

# Use the MDwarfFlareMetric detection criteria
metric_multi = MDwarfFlareMetric(metricName='multi_detect', outputLc=False)
metric_epoch = MDwarfFlareMetric(metricName='epoch_detect', outputLc=False)
metric_flare_complexity = MDwarfFlareMetric(metricName='flare_complexity', outputLc=False)

# Configure metric bundle
sql = ''

# Define plotDict for each metric separately
plotDict_multi = {'reduceFunc': np.sum, 'nside': 64, 'colorMin': 0, 'title': f'{runName} : MDwarfFlareMetric_MultiDetect'}
plotDict_epoch = {'reduceFunc': np.sum, 'nside': 64, 'colorMin': 0, 'title': f'{runName} : MDwarfFlareMetric_EpochDetect'}
plotDict_flare = {'reduceFunc': np.sum, 'nside': 64, 'colorMin': 0, 'title': f'{runName} : MDwarfFlareMetric_FlareComplexity'}

plotFuncs = [plots.HealpixSkyMap()]
summaryMetrics = [metrics.MeanMetric(maskVal=0)]

# Define metric bundles
bundles = [
    metricBundles.MetricBundle(metric_multi, slicer, sql, runName=runName,
                               plotDict=plotDict_multi, plotFuncs=plotFuncs, summaryMetrics=summaryMetrics,
                               fileRoot=f"{runName}_MDwarfFlareMetric_MultiDetect"),
    
    metricBundles.MetricBundle(metric_epoch, slicer, sql, runName=runName,
                               plotDict=plotDict_epoch, plotFuncs=plotFuncs, summaryMetrics=summaryMetrics,
                               fileRoot=f"{runName}_MDwarfFlareMetric_EpochDetect"),
    
    metricBundles.MetricBundle(metric_flare_complexity, slicer, sql, runName=runName,
                               plotDict=plotDict_flare, plotFuncs=plotFuncs, summaryMetrics=summaryMetrics,
                               fileRoot=f"{runName}_MDwarfFlareMetric_FlareComplexity"),
]

# Convert list to dictionary for MetricBundleGroup
bd = metricBundles.makeBundlesDictFromList(bundles)

# Print debugging information
print("DEBUG: Metrics in bgroup")
for key in bd.keys():
    print(f" - {key}")

# Define `bgroup` before calling `bgroup.runAll()`
bgroup = metricBundles.MetricBundleGroup(bd, opsdb, outDir=outDir, resultsDb=resultsDb)

bgroup.runAll()
bgroup.plotAll(closefigs=True)  # Prevents excessive figures from staying open

# Collect results
with open(outfile, "a") as out:
    for key in bd:
        if "USER" in key:
            continue
        found = np.size(np.where(bd[key].metricValues == 1)[0])
        out.write(f"{dmin}, {dmax}, {key}, {found}, {n_events}, {found/n_events}\n")


Simulating 100000 M Dwarf Flares with latitude-dependent weighting.
Final sample: 31234 flares (after latitude weighting)
Total M Dwarf Flare events simulated: 31234
First few peak times (days): [ 574.40406904  349.70250448  504.61601755 1729.70978611 3230.43371614]
DEBUG: Metrics in bgroup
 - baseline_v4.3.1_10yrs_MDwarfFlareMetric_MultiDetect
 - baseline_v4.3.1_10yrs_MDwarfFlareMetric_EpochDetect
 - baseline_v4.3.1_10yrs_MDwarfFlareMetric_FlareComplexity
DEBUG [0]: multi_detect type = <class 'int'>
DEBUG [0]: epoch_detect type = <class 'int'>
DEBUG [0]: flare_complexity type = <class 'float'>
DEBUG [0]: multi_detect type = <class 'int'>
DEBUG [0]: epoch_detect type = <class 'int'>
DEBUG [0]: flare_complexity type = <class 'float'>
DEBUG [0]: multi_detect type = <class 'int'>
DEBUG [0]: epoch_detect type = <class 'int'>
DEBUG [0]: flare_complexity type = <class 'float'>
DEBUG [1]: multi_detect type = <class 'int'>
DEBUG [1]: epoch_detect type = <class 'int'>
DEBUG [1]: flare_complexit

/opt/conda/envs/rubin-sims-v0.13.0/lib/python3.10/site-packages/rubin_sim/maf/plots/plotHandler.py:584: UserWarning: Cannot plot object metric values with this plotter.
  warnings.warn(
